In [0]:
import tensorflow as tf
import os

In [2]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

Cloning into 'fenwicks'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 487 (delta 40), reused 52 (delta 22), pack-reused 413
Receiving objects: 100% (487/487), 193.47 KiB | 3.58 MiB/s, done.
Resolving deltas: 100% (256/256), done.


In [0]:
BUCKET = 'gs://gs_colab' #@param {type:"string"}
PROJECT = 'tf_flowers' #@param {type:"string"}
MODEL = "ResNet50" #@param ["InceptionResNetV2", "ResNet50", "ResNet50V2", "InceptionV3", "MobileNetV2", "Xception"]

BATCH_SIZE = 128 #@param {type: "integer"}
EPOCHS = 20 #@param {type:"integer"}


In [4]:
fw.colab_tpu.setup_gcs()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)

In [0]:
base_model_func, weight_dir, weight_vars, img_size, normalizer = fw.keras_models.get_model(MODEL, BUCKET)

In [0]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.FLOWER_PHOTOS, './flower_photos')

In [0]:
# todo: show a random photo from each subdir

In [9]:
data_dir_local = os.path.join(data_dir_local, 'flower_photos')
data_dir_local

'./flower_photos/datasets/flower_photos'

In [0]:
data_fn = os.path.join(data_dir, 'all.tfrecord')

paths, y, labels = fw.io.data_dir_tfrecord(data_dir_local, data_fn, shuffle=True)
n_classes = len(labels)
n_all = len(y)

In [0]:
def build_nn():
  base_model = base_model_func()
  model = fw.Sequential()
  model.add(base_model)  
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(n_classes, use_bias=False))
  return model

In [0]:
n_valid = n_all // 5 // 8 * 8
n_train = n_all - n_valid
total_steps = n_train // BATCH_SIZE * EPOCHS

In [0]:
opt_func = fw.train.adam_sgdr_one_cycle(total_steps)
model_func = fw.tpuest.get_clf_model_func(build_nn, opt_func)

In [14]:
parser_train = fw.io.get_tfexample_image_parser(img_size, img_size, training=True, normalizer=normalizer)
parser_eval = fw.io.get_tfexample_image_parser(img_size, img_size, training=False, normalizer=normalizer)


est = fw.tpuest.get_tpu_estimator(n_train, n_valid, model_func, work_dir, 
                             weight_dir, weight_vars, BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/tf_flowers/2019-04-10-18:48:12', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.84.91.18:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feb750d70b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.84.91.18:8470', '_evaluation_master': 'grpc://10.84.91.18:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=22, num_shards=None, 

In [15]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 0, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 0, training=False)

est.train(train_input_func, steps=total_steps)
result0 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Querying Tensorflow master (grpc://10.84.91.18:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 12094140865951980020)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 330122869396192216)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3350169301198750023)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8156290480069505914)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 12266760561040780093)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/j

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[

In [16]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 1, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 1, training=False)

est.train(train_input_func, steps=total_steps)
result1 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [17]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 2, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 2, training=False)

est.train(train_input_func, steps=total_steps)
result2 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [18]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 3, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 3, training=False)

est.train(train_input_func, steps=total_steps)
result3 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [19]:
train_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_train,
  params['batch_size'], n_folds=5, val_fold_idx = 4, training=True)
valid_input_func = lambda params: fw.io.tfrecord_ds(data_fn, parser_eval, 
  params['batch_size'], n_folds=5, val_fold_idx = 4, training=False)

est.train(train_input_func, steps=total_steps)
result4 = est.evaluate(valid_input_func, steps=1)
fw.io.create_clean_dir(work_dir)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='gs://gs_colab/model/ResNet50/keras', vars_to_warm_start=['res5c_branch2c/bias[^/]', 'res5c_branch2b/kernel[^/]', 'res5c_branch2b/bias[^/]', 'res5c_branch2a/kernel[^/]', 'res5a_branch2a/bias[^/]', 'res5a_branch1/bias[^/]', 'res4f_branch2c/bias[^/]', 'res4f_branch2b/kernel[^/]', 'res4f_branch2a/bias[^/]', 'res4e_branch2a/bias[^/]', 'res4d_branch2c/kernel[^/]', 'res4d_branch2c/bias[^/]', 'res4d_branch2b/kernel[^/]', 'res4d_branch2b/bias[^/]', 'res4d_branch2a/kernel[^/]', 'res4d_branch2a/bias[^/]', 'res4c_branch2c/kernel[^/]', 'res4c_branch2c/bias[^/]', 'res4c_branch2b/kernel[^/]', 'res4c_branch2b/bias[^/]', 'res4c_branch2a/bias[^/]', 'res4b_branch2b/bias[^/]', 'res4a_branch2c/kernel[^/]', 'res4a_branch2c/bias[^/]', 'res4a_branch2b/kernel[^/]', 'res4a_branch2b/bias[^/]', 'res4a_branch2a/bias[^/]', 'res4a_branch1/

In [20]:
result0, result1, result2, result3, result4

({'accuracy': 0.9065934, 'global_step': 440, 'loss': 0.3669414},
 {'accuracy': 0.9230769, 'global_step': 440, 'loss': 0.257127},
 {'accuracy': 0.9120879, 'global_step': 440, 'loss': 0.26218334},
 {'accuracy': 0.9230769, 'global_step': 440, 'loss': 0.22019263},
 {'accuracy': 0.89972526, 'global_step': 440, 'loss': 0.25353155})